In [10]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import pandas as pd

In [11]:
# open the url
html = urlopen("https://www.yourinvestmentpropertymag.com.au/top-suburbs/vic/")
bsobj = BeautifulSoup(html, "lxml")

In [12]:
#find suburb's href
n=0
suburb_href = []
for l in bsobj.find_all('a', href=re.compile("^(/top-suburbs/)((?!:).)*$")):
    if 'href' in l.attrs:
        n = n +1
        suburb_href.append(l.attrs['href'])
    if n == 9:
        break
suburb_href.pop(0)
state =[]
for states in suburb_href:
    state.append(states.split('/')[-2])
state

['nsw', 'wa', 'act', 'vic', 'sa', 'tas', 'qld', 'nt']

In [13]:
labels =['state',
 'suburb',
 'postcode',
 'url',
 'Median_price_house',
 'Median_price_unit',
 'Quarterly_growth_house',
 'Quarterly_growth_unit'
 'twelve_month_growth_house',
 'twelve_month_growth_unit',
 'Average_annual_growth_house',
 'Average_annual_growth_unit',
 'Weekly_median_rent_house',
 'Weekly_median_rent_unit',
 'Gross_rental_yield_house',
 'Gross_rental_yield_unit',
 'Number_of_Sales_(12m)_house',
 'Number_of_Sales_(12m)_unit',
 'Avg._Days_on_Market_(12m)_house',
 'Avg._Days_on_Market_(12m)_unit',
 'Total_population_2011',
 'Total_population_2016',
 'Population_change_(5y)_2011',
 'Population_change_(5y)_2016',
 'Median_household_income_(p/w)_2011',
 'Median_household_income_(p/w)_2016',
 'Household_income_change_(5y)_2011',
 'Household_income_change_(5y)_2016',
 'Median_age_of_persons_2011',
 'Median_age_of_persons_2016',
 'DESC_house',
 'DESC_unit']

In [14]:
df = pd.DataFrame(columns=labels)

In [15]:
home_link = 'https://www.yourinvestmentpropertymag.com.au'
i= 0 

for n in range(len(suburb_href)):
    state_url = home_link + suburb_href[n]
    response = requests.get(state_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    for href in soup.find_all('a', class_="visible-item"):

        if len(state[n]) == 3:
            suburb_url = state_url + href['href'][17:]

            df.loc[i,'state'] = state[n]
            df.loc[i,'url'] = suburb_url
            df.loc[i,'postcode'] = href['href'][17:].split('-')[0]
            df.loc[i,'suburb'] = href['href'][17:].split('-',1)[1:][0].replace('-',' ')
            suburb_response = requests.get(suburb_url)
            suburb_soup = BeautifulSoup(suburb_response.content, 'html.parser')
            c = 4
            x = 19
            for t in suburb_soup.find('div', {'class' : 'key-market-data'}).find_all('span'):
                data = t.text.replace("$","")
                data = data.replace(",","")
                data = data.replace("+","")
                if data == '–':
                    data = 'Null'
                df.loc[i,labels[c]] = data
                c= c+1
            for t in suburb_soup.find('div', {'class' : 'key-demographics'}).find_all('span'):
                data = t.text.replace("$","")
                data = data.replace(",","")
                data = data.replace("+","")
                if data == '–':
                    data = 'Null'
                df.loc[i,labels[x]] = data
                x= x+1
        else:
            suburb_url = state_url + href['href'][16:]
            df.loc[i,'state'] = state[n]
            df.loc[i,'url'] = suburb_url
            df.loc[i,'postcode'] = href['href'][16:].split('-')[0]
            df.loc[i,'suburb'] = href['href'][16:].split('-',1)[1:][0].replace('-',' ')
            suburb_response = requests.get(suburb_url)
            suburb_soup = BeautifulSoup(suburb_response.content, 'html.parser')
            c = 4
            x = 19
            for t in suburb_soup.find('div', {'class' : 'key-market-data'}).find_all('span'):
                data = t.text.replace("$","")
                data = data.replace(",","")
                data = data.replace("+","")
                if data == '–':
                    data = 'Null'
                df.loc[i,labels[c]] = data
                c= c+1
            for t in suburb_soup.find('div', {'class' : 'key-demographics'}).find_all('span'):
                data = t.text.replace("$","")
                data = data.replace(",","")
                data = data.replace("+","")
                if data == '–':
                    data = 'Null'
                df.loc[i,labels[x]] = data
                x= x+1


        i = i +1 



In [16]:
for i in range(len(df)):
    suburb_url = df['url'][i]
    suburb_response = requests.get(suburb_url)
    suburb_desc_soup = BeautifulSoup(suburb_response.content, 'html.parser')
    DESC_house = suburb_desc_soup.find('div', {'aria-labelledby' :'pills-house-report-tab'}).text.replace('\n','')
    df.loc[i,'DESC_house'] = DESC_house
    DESC_unit = suburb_desc_soup.find('div', {'id' :'pills-unit-report'}).text.replace('\n','')
    df.loc[i,'DESC_unit'] = DESC_unit

In [21]:
df.to_csv('suburb.csv')  